In [1]:
#r "nuget:Microsoft.DotNet.Interactive.AI, *-*"

Installed Packages Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23468.5

Loading extension script from `C:\Users\dicolomb\.nuget\packages\microsoft.dotnet.interactive.ai\1.0.0-beta.23468.5\interactive-extensions\dotnet\extension.dib`

Added magic commands
 - `#!connect azure-openai`
 - `#!connect openai`
 - `#!connect huggingface`

Loading extensions from `C:\Users\dicolomb\.nuget\packages\skiasharp\2.88.5\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extension script from `C:\Users\dicolomb\.nuget\packages\duckdb.interactiveextension\1.0.108\interactive-extensions\dotnet\extension.dib`

Query DuckDB databases. 
 This extension adds support for connecting to DuckDB databases using the #!connect duckdb magic command. ADO.NET support via DuckDB.NET project

In [2]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [3]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [4]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

Kernel added: #!knowledge(duckDbVectorRAW)

Kernel added: #!knowledge(duckDbVector)

Kernel added: #!knowledge

In [5]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gtp-35-turbo --use-knowledge knowledge

Kernel added: #!chat(skill)

Kernel added: #!chat(text)

Kernel added: #!chat

### we will be using text embeddings and cosine distance similarity to focus the conversation with the agent on the questions the user is about ot submit

In [6]:
#r "nuget: Microsoft.ML, 3.0.0-preview.23266.6"

Installed Packages Microsoft.ML, 3.0.0-preview.23266.6

In [7]:
using Microsoft.ML;
using Microsoft.ML.Data;

## Focus the conversation

At each turn let's filter out irrelevant turns for the question at hand. TextEmbedding ans cosine similarity will help us keep the conversation on track!

In [8]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.AI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.Tokenizers;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;

var chatCompletionKernel = Microsoft.DotNet.Interactive.Kernel.Root.FindKernelByName("chat") as ChatCompletionKernel;
var knowledgeKernel = Microsoft.DotNet.Interactive.Kernel.Root.FindKernelByName("knowledge") as KnowledgeKernel;

chatCompletionKernel.SetUserTurnFilter(async (turn, userPrompt, token) =>{
    var userEmbeddings = await knowledgeKernel.GenerateEmbeddingsAsync(userPrompt, token);
    var questionRelevance = userEmbeddings
                    .CosineSimilarity(turn.QuestionTexEmbedding);
    var answerRelevance = userEmbeddings
                    .CosineSimilarity(turn.AnswerTexEmbedding);
    var relevanceThreshold = 0.8;
    if (questionRelevance >= relevanceThreshold || answerRelevance >= relevanceThreshold)
    {
        return true;
    }else{
        //"The turn is not relevant to the conversation.".Display();
        //turn.Display();
        return false;
    }
});

enrich the prompt using the knowledge kernel

In [9]:
chatCompletionKernel.SetPromptTransform(async (role, prompt, _) => {
    
    if(role != AuthorRole.User){
        return prompt;
    }
    var queryCommand = new QueryKnowledge(prompt, 5);
    knowledgeKernel.SetSuppressDisplay(true);
    var result = await knowledgeKernel.SendAsync(queryCommand);
    knowledgeKernel.SetSuppressDisplay(false);
    var searchResults = result.Events.OfType<ReturnValueProduced>().Single(e => e.Command.TargetKernelName == knowledgeKernel.Name).ToKnowledgeQueryResults();

    if(searchResults.Length == 0){
        return prompt;
    }

    return $"""
    Given the fatcs:
    {string.Join("\n", searchResults.OrderBy(r=> r.Relevance).Select(r => r.Text))}

    Then, {prompt}
    """;
});

## what are we talking about??

Let's tap into ML and use K-Means to cluster the chatlog using the text embeddings of each turns!

In [10]:
public class ModelInput
{
  public string Text {get;set;}
  [VectorType(1536)]
  public float[] Embedding {get;set;}
}

In [11]:
IDataView clusteredData = null;

chatCompletionKernel.OnConversationTurnCompleted(( chatLog, toekn) =>
{
   var embeddings = new List<ModelInput>();
   var ctx = new MLContext();
   var clusterCount  = Math.Min(20, 2+ (chatLog.Turns.Count / 10));
   if(chatLog.Turns.Count < clusterCount)
   {
      return Task.CompletedTask;
   }  
   foreach (var turn in chatLog.Turns)
   {
      switch (turn){
       case ChatLog.ChatUserTurn  userTurn:
          embeddings.Add(new ModelInput() { Text = userTurn.Question, Embedding = userTurn.QuestionTexEmbedding});
          embeddings.Add(new ModelInput() { Text = userTurn.Answer, Embedding = userTurn.AnswerTexEmbedding });
          break;
          case ChatLog.ChatSystemTurn systemTurn:
         // embeddings.Add(new ModelInput() { Text = systemTurn.Prompt, Embedding = systemTurn..AnswerTexEmbedding.AsReadOnlySpan().ToArray() });
          break;
      }
   }
   var idv = ctx.Data.LoadFromEnumerable(embeddings);

   var pipeline = 
   ctx.Clustering.Trainers.KMeans("Embedding", numberOfClusters: clusterCount);
   clusteredData = pipeline.Fit(idv).Transform(idv);
   return Task.CompletedTask;
});

## Start chatting then!

In [12]:
I would like to ask you questions about coding in C#

Sure, I'll do my best to answer your questions about coding in C#. What would you like to know?

In [13]:
How do I cook a fish?

As an AI language model, I don't eat or cook food, but here are some general steps to cook a fish:

1. Clean the fish: Remove the scales, gut the fish, and rinse it thoroughly with cold water.

2. Season the fish: Rub the fish with salt, pepper, and any other desired seasonings.

3. Choose a cooking method: You can bake, grill, fry, or steam the fish.

4. Preheat the oven or grill: If you're baking or grilling the fish, preheat the oven or grill to the desired temperature.

5. Cook the fish: Place the fish on a baking sheet or grill and cook until it's done. The cooking time will depend on the size and thickness of the fish.

6. Check for doneness: The fish is done when it flakes easily with a fork and is no longer translucent.

7. Serve: Serve the fish with your favorite sides and enjoy!

In [14]:
What is a c# class?

In C#, a class is a blueprint or a template for creating objects that share common properties, methods, and events. It defines the structure and behavior of an object, including its data members (fields), methods, properties, and events. 

A class can be thought of as a user-defined data type that encapsulates data and behavior. It provides a way to organize and modularize code, making it easier to maintain and reuse. 

For example, if you were creating a program to manage a library, you might define a class called "Book" that contains properties such as "Title", "Author", and "ISBN", as well as methods for checking out and returning books. You could then create multiple instances of the "Book" class to represent different books in the library.

In [15]:
What is the most common modal scale in heavy metal music?

The most common modal scale in heavy metal music is the Phrygian mode.

In [16]:
How do I setup my amplifier to play heavy metal guitar riffs?

To set up your amplifier to play heavy metal guitar riffs, follow these steps:

1. Start by selecting a high-gain channel on your amplifier. This will give you the distortion and overdrive necessary for heavy metal riffs.

2. Adjust the gain knob to your desired level. For heavy metal, you'll want a high gain setting to get that crunchy, distorted sound.

3. Adjust the bass, mid, and treble knobs to your liking. For heavy metal, you'll typically want to boost the bass and treble while cutting the mids to get a more aggressive, cutting tone.

4. Experiment with the presence and resonance knobs if your amplifier has them. These can help you fine-tune your tone and get the right amount of clarity and punch.

5. Finally, adjust the volume to your desired level. Be careful not to crank it too loud, as this can damage your hearing and your amplifier.

Once you've set up your amplifier, you can start playing heavy metal riffs by using power chords, palm muting, and fast picking techniques. Re

In [17]:
What is the mest way to model embeddings in a relational database?

There are several ways to model embeddings in a relational database, but one of the most common approaches is to use a table with columns for each dimension of the embedding. For example, if the embedding has 100 dimensions, the table would have 100 columns. Each row in the table would represent an entity or object that has an embedding, and the values in the columns would represent the values of the embedding dimensions for that entity.

Another approach is to use a separate table for each dimension of the embedding, with each table containing a single column for the values of that dimension. This approach can be more flexible and scalable, as it allows for adding or removing dimensions without having to modify the main table. However, it can also be more complex to query and join the data across multiple tables.

Ultimately, the best way to model embeddings in a relational database depends on the specific requirements and constraints of the application, as well as the size and comple

In [18]:
What is a motorbike?

A motorbike, also known as a motorcycle or bike, is a two-wheeled vehicle powered by an engine. It typically has a seat for the rider, handlebars for steering, and footrests for the rider's feet. Motorbikes come in various sizes and styles, including sport bikes, cruisers, touring bikes, and dirt bikes. They are commonly used for transportation, recreation, and racing.

In [19]:
What is the best way to model a motorbike in C#?

Modeling a motorbike in C# would require the use of a 3D graphics library or game engine that supports C#. Here are the general steps to model a motorbike in C#:

1. Choose a 3D graphics library or game engine that supports C#, such as Unity, Unreal Engine, or OpenTK.
2. Create a 3D model of the motorbike using a 3D modeling software, such as Blender or Maya.
3. Import the 3D model into the chosen graphics library or game engine.
4. Apply textures and materials to the 3D model to give it a realistic appearance.
5. Add physics and collision detection to the motorbike model to simulate its movement and interaction with the environment.
6. Implement user input controls to allow the player to control the motorbike's movement.
7. Test and refine the motorbike model to ensure it functions correctly and provides an enjoyable user experience.

Overall, modeling a motorbike in C# requires a combination of 3D modeling skills, programming knowledge, and game development experience.

In [20]:
Show me an example of a class that represents a motorbike using C#

Sure, here's an example of a class that represents a motorbike using C#:

```
public class Motorbike
{
    // Fields
    private string make;
    private string model;
    private int year;
    private float engineSize;
    private bool isRunning;

    // Constructor
    public Motorbike(string make, string model, int year, float engineSize)
    {
        this.make = make;
        this.model = model;
        this.year = year;
        this.engineSize = engineSize;
        this.isRunning = false;
    }

    // Properties
    public string Make
    {
        get { return make; }
        set { make = value; }
    }

    public string Model
    {
        get { return model; }
        set { model = value; }
    }

    public int Year
    {
        get { return year; }
        set { year = value; }
    }

    public float EngineSize
    {
        get { return engineSize; }
        set { engineSize = value; }
    }

    public bool IsRunning
    {
        get { return isRunning; }
    }

    /

Load knowledge from disk

In [23]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AI.DuckDb;
using System.IO;
var currentDirectory = Directory.GetCurrentDirectory();
var dataFile = Path.Combine(currentDirectory, "interactive_knowledge.json");

In [25]:
await Microsoft.DotNet.Interactive.Kernel.Root.SendAsync(new ImportCollection("defaultVectorCollection", dataFile, "knowledge(duckDbVector)"));

In [27]:
Can I implement a chatbot using C# in .NET interactive using polyglot Notebooks?

Yes, it is possible to implement a chatbot using C# in .NET Interactive using Polyglot Notebooks. .NET Interactive supports C# as one of its languages, and the Polyglot Notebooks extension allows for multiple languages to be used in one notebook. You can use C# to write the code for your chatbot and integrate it with other languages such as Python or JavaScript for additional functionality. Additionally, you can use the rich visualization capabilities of .NET Interactive to create interactive chatbot interfaces.

In [14]:
chatCompletionKernel.ResetChatLog();

In [15]:
clusteredData.Preview().Display();

4 columns, 4 rows Schema [ Text: String, Embedding: Vector<Single, 1536>, PredictedLabel: Key<UInt32, 0-1> {ScoreColumnKind, ScoreValueKind, ScoreColumnSetId}, Score: Vector<Single, 2> {ScoreColumnKind, ScoreColumnSetId, ScoreValueKind} ] Count 4 (values) index value 0 Text: String Name Text Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 1 Embedding: Vector<Single, 1536> Name Embedding Index 1 IsHidden False Type Vector<Single, 1536> Dimensions [ 1536 ] IsKnownSize True ItemType Single RawType System.Single Size 1536 RawType Microsoft.ML.Data.VBuffer<System.Single> Annotations Schema [ ] Count 0 (values) (empty) 2 PredictedLabel: Key<UInt32, 0-1> {ScoreColumnKind, ScoreValueKind, ScoreColumnSetId} Name PredictedLabel Index 2 IsHidden False Type Key<UInt32, 0-1> Count 2 RawType System.UInt32 Annotations ScoreColumnKind, ScoreValueKind, ScoreColumnSetId Schema [ ScoreColumnKind: String, ScoreValueKind: String, ScoreColumnSetId: Key<UInt32, 0-2147483646> ] Count 3 (values) index value 0 ScoreColumnKind: String Name ScoreColumnKind Index 0 IsHidden False Type String Annotations 1 ScoreValueKind: String Name ScoreValueKind Index 1 IsHidden False Type String Annotations 2 ScoreColumnSetId: Key<UInt32, 0-2147483646> Name ScoreColumnSetId Index 2 IsHidden False Type Key Annotations 3 Score: Vector<Single, 2> {ScoreColumnKind, ScoreColumnSetId, ScoreValueKind} Name Score Index 3 IsHidden False Type Vector<Single, 2> Dimensions [ 2 ] IsKnownSize True ItemType Single RawType System.Single Size 2 RawType Microsoft.ML.Data.VBuffer<System.Single> Annotations ScoreColumnKind, ScoreColumnSetId, ScoreValueKind Schema [ ScoreColumnKind: String, ScoreColumnSetId: Key<UInt32, 0-2147483646>, ScoreValueKind: String ] Count 3 (values) index value 0 ScoreColumnKind: String Name ScoreColumnKind Index 0 IsHidden False Type String Annotations 1 ScoreColumnSetId: Key<UInt32, 0-2147483646> Name ScoreColumnSetId Index 1 IsHidden False Type Key Annotations 2 ScoreValueKind: String Name ScoreValueKind Index 2 IsHidden False Type String Annotations ColumnView index value 0 Text: String Column Text: String Name Text Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) Values index value 0 I would like to ask you questions about coding in C# 1 Sure, I'll do my best to answer your questions about coding in C#. What would you like to know? 2 How do I cook a fish? 3 As an AI language model, I don't eat or cook, but here are some general steps to cook a fish:

1. Clean the fish: Rinse the fish under cold water and pat it dry with paper towels. Remove any scales, guts, and fins.

2. Season the fish: Rub the fish with salt, pepper, and any other desired seasonings.

3. Choose a cooking method: There are several ways to cook fish, including baking, grilling, frying, and steaming.

4. Cook the fish: Follow the instructions for your chosen cooking method. Generally, fish should be cooked until it is opaque and flakes easily with a fork.

5. Serve the fish: Once the fish is cooked, remove it from the heat and let it rest for a few minutes. Serve it with your favorite sides and enjoy! 1 Embedding: Vector<Single, 1536> Column Embedding: Vector<Single, 1536> Name Embedding Index 1 IsHidden False Type Vector<Single, 1536> Dimensions [ 1536 ] IsKnownSize True ItemType Single RawType System.Single Size 1536 RawType Microsoft.ML.Data.VBuffer<System.Single> Annotations Schema [ ] Count 0 (values) (empty) Values index value 0 Dense vector of size 1536 IsDense True Length 1536 1 Dense vector of size 1536 IsDense True Length 1536 2 Dense vector of size 1536 IsDense True Length 1536 3 Dense vector of size 1536 IsDense True Length 1536 2 PredictedLabel: Key<UInt32, 0-1> Column PredictedLabel: Key<UInt32, 0-1> {ScoreColumnKind, ScoreValueKind, ScoreColumnSetId} Name PredictedLabel Index 2 IsHidden False Type Key<UInt32, 0-1> Count 2 Ra